# Stop Data
This file will read in the csv data from a file, 
clean and prepare it 
enter it in the stops database

In [11]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

# Read data from csv file 

In [12]:
stop_file = "copstop/static/data/Police_Stop_data2.csv"
stop_df=pd.read_csv(stop_file)
stop_df.tail()

C:\Users\blizz\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,OBJECTID,callDisposition,citationIssued,gender,lastUpdateDate,lat,long,masterIncidentNumber,neighborhood,personSearch,policePrecinct,preRace,problem,race,reason,responseDate,vehicleSearch,x,y
109037,109038,GOA-Gone on Arrival,NO,Unknown,1546417236000,45.044855,-93.311791,19-001106,Shingle Creek,NO,4.0,Unknown,Suspicious Vehicle (P),Unknown,Citizen / 9-1-1,2019-01-01 19:34:06.000000,NO,-1.038742e+07,5.628586e+06
109038,109039,ADV-Advised,NO,Female,1546417237000,45.003070,-93.261570,19-001107,Sheridan,NO,2.0,Unknown,Traffic Law Enforcement (P),Black,Equipment Violation,2019-01-01 19:36:27.000000,NO,-1.038183e+07,5.622005e+06
109039,109040,ADV-Advised,NO,Male,1546417237000,44.962730,-93.253960,19-001114,Ventura Village,NO,3.0,Unknown,Traffic Law Enforcement (P),White,Moving Violation,2019-01-01 19:47:29.000000,NO,-1.038098e+07,5.615656e+06
109040,109041,ADV-Advised,NO,Male,1546417236000,44.937771,-93.268239,19-001139,Central,NO,3.0,Latino,Suspicious Person (P),White,Citizen / 9-1-1,2019-01-01 20:48:46.000000,NO,-1.038257e+07,5.611730e+06
109041,109042,UTL-Unable to Locate,NO,Male,1546417236000,45.001102,-93.313329,19-001159,Willard - Hay,NO,4.0,White,Suspicious Person (P),Unknown,Citizen / 9-1-1,2019-01-01 21:52:06.000000,NO,-1.038759e+07,5.621695e+06


# Imports for database


In [13]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Set up tables

In [14]:
# Define our neighborhoodData table
class neighborhoodData(Base):
    __tablename__ = 'neighborhoodData'
    id = Column(Integer, primary_key=True)
    neighborhood = Column(String)
    totalPop = Column(Integer)
    margin = Column(Integer)
    stopCnt=Column(Integer)

In [15]:
# Define our stopData table
class stopData(Base):
    __tablename__ = 'stopData'
    OBJECTID = Column(Integer, primary_key=True)
    neighborhood = Column(String)
    responseDate = Column(String)
    citationIssued = Column(String)
    lat= Column(Float)
    lon= Column(Float)
    gender =Column(String)
    responseDow = Column(Integer)
    responseDay =Column(Integer)
    responseMonth = Column(Integer)
    responseMonthName =Column(String)
    responseYear = Column(Integer)
    

In [16]:
# Define dowData table
class dowData(Base):
    __tablename__ = 'dowData'
    neighborhood = Column((String), primary_key=True)
    gender =Column(String)
    responseDow = Column(Integer)
    genderCount = Column(Integer)

In [17]:
# Define our citationData table
class citationData(Base):
    __tablename__ = 'citationData'
    responseDay =Column((Integer),primary_key=True)
    citationCnt = Column(Integer)
    

In [18]:
# tables only in python
Base.metadata.tables
# Create our database engine
engine = create_engine('sqlite:///copstop/db/stops.sqlite')
# This is where we create our tables in the database
Base.metadata.create_all(engine)
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

# Reduce Columns
Reduce the columns to those needed and eliminate rows with null gender  
check count to confirm later operations  

In [19]:
      #select needed columns
select_df=stop_df.loc[:,["OBJECTID", "responseDate","citationIssued","lat","long","gender","neighborhood"]]

# remove null gender rows
reducedA_df=select_df.dropna(axis=0,subset=['gender'])

# remove null neighborhood rows
reduced_df=reducedA_df.dropna(axis=0,subset=['neighborhood'])
reduced_df.head()

#column name change from long to lon
reduced_df=reduced_df.rename(columns={'long':'lon'})
reduced_df.head()

,OBJECTID,responseDate,citationIssued,lat,lon,gender,neighborhood
0,1,2016-10-31 17:40:47.000000,NaN,44.979570,-93.272570,Male,Downtown West
1,2,2016-10-31 18:06:36.000000,NaN,44.962689,-93.275921,Male,Steven's Square - Loring Heights
2,3,2016-10-31 18:20:54.000000,NaN,45.024836,-93.288069,Unknown,Webber - Camden
3,4,2016-10-31 18:23:20.000000,NaN,44.946560,-93.247410,Male,Corcoran
4,5,2016-10-31 18:26:05.000000,NaN,44.906170,-93.255010,Male,Hale


# Parse date
responseDow = day of the week  
responseDay = day of the month as zero padded decimal  
responseDonth = month as a zero padded decimal  
responseMonthName= month as abbreviated name  
responseYear  

In [20]:
#parse date
date_list= [datetime.strptime( entry,"%Y-%m-%d %H:%M:%S.%f") for entry in reduced_df["responseDate"]]
# determine response day of week
dow_list= [datetime.strftime(entry,"%w") for entry in date_list]

# split off response day, month and year
day_list = [datetime.strftime(entry,"%d") for entry in date_list]
month_list=[datetime.strftime(entry,"%m") for entry in date_list]
month_name_list=[datetime.strftime(entry,"%b") for entry in date_list]
year_list= [datetime.strftime(entry,"%Y") for entry in date_list]

#add columns to reduced_df
reduced_df.loc[:,"responseDow"]=dow_list
reduced_df.loc[:,"responseDay"]=day_list
reduced_df.loc[:,"responseMonth"]=month_list
reduced_df.loc[:,"responseMonthName"]=month_name_list
reduced_df.loc[:,"responseYear"]=year_list

#keep just 2018
reduced_df=reduced_df.loc[reduced_df["responseYear"]== '2018']
reduced_df

,OBJECTID,responseDate,citationIssued,lat,lon,gender,neighborhood,responseDow,responseDay,responseMonth,responseMonthName,responseYear
60846,60847,2018-01-01 02:29:04.000000,NO,45.008491,-93.314608,Unknown,Willard - Hay,1,01,01,Jan,2018
60847,60848,2018-01-01 05:00:03.000000,YES,44.999820,-93.267670,Male,Sheridan,1,01,01,Jan,2018
60848,60849,2018-01-01 06:04:47.000000,NO,44.972810,-93.216582,Male,Prospect Park - East River Road,1,01,01,Jan,2018
60849,60850,2018-01-01 06:35:08.000000,NO,44.950810,-93.276600,Male,Whittier,1,01,01,Jan,2018
60850,60851,2018-01-01 06:56:16.000000,NO,45.014069,-93.256723,Male,Holland,1,01,01,Jan,2018
60851,60852,2018-01-01 07:47:48.000000,NO,45.033390,-93.312360,Male,Victory,1,01,01,Jan,2018
60852,60853,2018-01-01 08:54:25.000000,NO,44.940441,-93.277916,Female,Lyndale,1,01,01,Jan,2018
60853,60854,2018-01-01 09:55:07.000000,YES,44.949260,-93.298280,Male,Lowry Hill East,1,01,01,Jan,2018
60854,60855,2018-01-01 09:59:08.000000,NO,44.990970,-93.258430,Male,St. Anthony West,1,01,01,Jan,2018
60856,60857,2018-01-01 11:41:09.000000,NO,45.015807,-93.289292,Unknown,McKinley,1,01,01,Jan,2018


# Get a list of neighborhoods
remove null neighborhood

In [21]:
#Get a listing and count of neighborhood stops
neighbor_df=reduced_df.groupby('neighborhood') ["neighborhood"].count()
neighborhood_df=pd.DataFrame(neighbor_df)
neighborhood_df=neighborhood_df.rename(columns={'neighborhood':'stopCnt'})

neighborhood_df=neighborhood_df.sort_values("neighborhood")
neighborhood_df.tail()

,stopCnt
neighborhood,
West Calhoun,41
Whittier,1843
Willard - Hay,1028
Windom,463
Windom Park,562


# Read in population data and merge with neighborhoods

In [22]:
pop_file = "copstop/static/data/population_data.csv"
pop_df=pd.read_csv(pop_file)
pop_df.tail(10)

,neighborhood,totalPop,margin
75,Ventura Village,6028,484
76,Victory,4550,279
77,Waite Park,5558,245
78,Webber - Camden,5296,322
79,Wenonah,4464,223
80,West Calhoun,1938,146
81,Whittier,14828,781
82,Willard - Hay,9345,538
83,Windom,4844,240
84,Windom Park,5381,279


In [23]:
merge_df=pd.merge(neighborhood_df,pop_df,on="neighborhood",how="left")
merge_df=merge_df[['neighborhood','totalPop','margin','stopCnt']]
merge_df.tail()


,neighborhood,totalPop,margin,stopCnt
82,West Calhoun,1938.0,146.0,41
83,Whittier,14828.0,781.0,1843
84,Willard - Hay,9345.0,538.0,1028
85,Windom,4844.0,240.0,463
86,Windom Park,5381.0,279.0,562


In [24]:
merge_df.to_csv("copstop/static/data/neighborhood_data.csv", index=False,header=True)

# Put neighborhood data in the table

In [25]:
merge_df.to_sql(neighborhoodData.__tablename__, engine, if_exists='replace')

session.commit()

In [26]:
engine.execute('select * from neighborhoodData').fetchall()

[(0, 'Armatage', 4915.0, 262.0, 73),
 (1, 'Audubon Park', 5007.0, 237.0, 671),
 (2, 'Bancroft', 3513.0, 231.0, 85),
 (3, 'Beltrami', 1142.0, 144.0, 225),
 (4, 'Bottineau', 1618.0, 170.0, 259),
 (5, 'Bryant', 3179.0, 324.0, 78),
 (6, 'Bryn - Mawr', 2798.0, 124.0, 116),
 (7, 'CARAG', 5723.0, 262.0, 405),
 (8, 'Camden Industrial', None, None, 36),
 (9, 'Cedar - Isles - Dean', 2901.0, 270.0, 83),
 (10, 'Cedar Riverside', 8576.0, 760.0, 523),
 (11, 'Central', 8737.0, 451.0, 500),
 (12, 'Cleveland', 2980.0, 213.0, 268),
 (13, 'Columbia Park', 1717.0, 91.0, 156),
 (14, 'Como', 7208.0, 590.0, 474),
 (15, 'Cooper', 3614.0, 199.0, 123),
 (16, 'Corcoran', 4239.0, 306.0, 282),
 (17, 'Diamond Lake', 5583.0, 224.0, 88),
 (18, 'Downtown East', 1711.0, 123.0, 303),
 (19, 'Downtown West', 7357.0, 482.0, 3500),
 (20, 'ECCO', 2497.0, 197.0, 222),
 (21, 'East Harriet', 3667.0, 158.0, 148),
 (22, 'East Isles', 3315.0, 222.0, 307),
 (23, 'East Phillips', 4743.0, 631.0, 968),
 (24, 'Elliot Park', 6616.0, 446

# modify Gender 
combine gender Unknown and Gender Non-Conforming into gender Other

In [27]:
#reduce warnings
pd.options.mode.chained_assignment = None  # default='warn'
#fill in Unknown vaue with Other
reduced_df.loc[reduced_df.gender=="Unknown","gender"] = "Other"
      # fill in Gender Non-Conforming values with other
reduced_df.loc[reduced_df.gender =="Gender Non-Conforming","gender"] = "Other"
reduced_df.tail()

,OBJECTID,responseDate,citationIssued,lat,lon,gender,neighborhood,responseDow,responseDay,responseMonth,responseMonthName,responseYear
108966,108967,2018-12-31 21:16:14.000000,NO,45.013150,-93.296850,Female,Jordan,1,31,12,Dec,2018
108967,108968,2018-12-31 22:42:49.000000,NO,44.999150,-93.292580,Male,Near - North,1,31,12,Dec,2018
108968,108969,2018-12-31 23:05:11.000000,NO,44.916070,-93.248580,Male,Northrop,1,31,12,Dec,2018
108969,108970,2018-12-31 23:18:22.000000,NO,44.938601,-93.293297,Male,CARAG,1,31,12,Dec,2018
108970,108971,2018-12-31 23:27:43.000000,NO,44.948370,-93.224320,Female,Longfellow,1,31,12,Dec,2018


In [28]:
#what gender values are present ?
#verify count
gender=reduced_df.groupby(["gender"]).count()
gender

,OBJECTID,responseDate,citationIssued,lat,lon,neighborhood,responseDow,responseDay,responseMonth,responseMonthName,responseYear
gender,,,,,,,,,,,
Female,8652,8652,8652,8652,8652,8652,8652,8652,8652,8652,8652
Male,25315,25315,25315,25315,25315,25315,25315,25315,25315,25315,25315
Other,5953,5953,5953,5953,5953,5953,5953,5953,5953,5953,5953


# Modify citationIssued 
change NaN to NO

In [29]:
# change null citationIssued to NO
reduced_df[["citationIssued"]]=reduced_df[["citationIssued"]].fillna(value='NO')
reduced_df

,OBJECTID,responseDate,citationIssued,lat,lon,gender,neighborhood,responseDow,responseDay,responseMonth,responseMonthName,responseYear
60846,60847,2018-01-01 02:29:04.000000,NO,45.008491,-93.314608,Other,Willard - Hay,1,01,01,Jan,2018
60847,60848,2018-01-01 05:00:03.000000,YES,44.999820,-93.267670,Male,Sheridan,1,01,01,Jan,2018
60848,60849,2018-01-01 06:04:47.000000,NO,44.972810,-93.216582,Male,Prospect Park - East River Road,1,01,01,Jan,2018
60849,60850,2018-01-01 06:35:08.000000,NO,44.950810,-93.276600,Male,Whittier,1,01,01,Jan,2018
60850,60851,2018-01-01 06:56:16.000000,NO,45.014069,-93.256723,Male,Holland,1,01,01,Jan,2018
60851,60852,2018-01-01 07:47:48.000000,NO,45.033390,-93.312360,Male,Victory,1,01,01,Jan,2018
60852,60853,2018-01-01 08:54:25.000000,NO,44.940441,-93.277916,Female,Lyndale,1,01,01,Jan,2018
60853,60854,2018-01-01 09:55:07.000000,YES,44.949260,-93.298280,Male,Lowry Hill East,1,01,01,Jan,2018
60854,60855,2018-01-01 09:59:08.000000,NO,44.990970,-93.258430,Male,St. Anthony West,1,01,01,Jan,2018
60856,60857,2018-01-01 11:41:09.000000,NO,45.015807,-93.289292,Other,McKinley,1,01,01,Jan,2018


In [30]:
# verify count
citation=reduced_df.groupby(["citationIssued"]).count()
citation

,OBJECTID,responseDate,lat,lon,gender,neighborhood,responseDow,responseDay,responseMonth,responseMonthName,responseYear
citationIssued,,,,,,,,,,,
NO,33550,33550,33550,33550,33550,33550,33550,33550,33550,33550,33550
YES,6370,6370,6370,6370,6370,6370,6370,6370,6370,6370,6370


# Deliver clean stop data
to file "static/data/clean_data.csv"  
or to database

In [31]:
reduced_df.to_csv("copstop/static/data/clean_stop_data.csv", index=False,header=True)

# Add Stop data to the database

In [32]:
reduced_df.to_sql(stopData.__tablename__, engine, if_exists='replace',chunksize=50)

session.commit()

In [33]:
#test for data
neighbor = session.query(stopData.neighborhood, stopData.lat, stopData.lon).filter_by(neighborhood="Como").first()
neighbor

('Como', 44.98435442, -93.22032932)

# Prep responseDow data
get count by neighborhood, responseDow and gender

In [34]:
# get count of genders by neighborhood and responseDow
group_df=reduced_df.groupby(["neighborhood","responseDow","gender"]).count()
group_df.head()

OBJECTID  responseDate  citationIssued  lat  \
neighborhood responseDow gender                                                
Armatage     0           Male           5             5               5    5   
                         Other          3             3               3    3   
             1           Female         1             1               1    1   
                         Male           3             3               3    3   
                         Other          8             8               8    8   

                                 lon  responseDay  responseMonth  \
neighborhood responseDow gender                                    
Armatage     0           Male      5            5              5   
                         Other     3            3              3   
             1           Female    1            1              1   
                         Male      3            3              3   
                         Other     8            8              8   

                                 responseMonthName  responseYear  
neighborhood responseDow gender                                   
Armatage     0           Male                    5             5  
                         Other                   3             3  
             1           Female                  1             1  
                         Male                    3             3  
                         Other                   8             8

In [35]:
# remove indexing rename columns and reduce columns
new_group=group_df.reset_index()
next_group=new_group.rename(columns={"OBJECTID":"genderCount"})
dow_group=next_group[["neighborhood","responseDow","gender","genderCount"]]
dow_group.head()

,neighborhood,responseDow,gender,genderCount
0,Armatage,0,Male,5
1,Armatage,0,Other,3
2,Armatage,1,Female,1
3,Armatage,1,Male,3
4,Armatage,1,Other,8


# Deliver clean day of week data  
to file "static/data/dow_clean_data.csv" 
or database

In [36]:
# write ad csv file -- json file doesn't come out right
dow_group.to_csv("copstop/static/data/clean_dow_data.csv" ,index=False, header=True)

# Add dow Data to database

In [37]:
dow_group.to_sql(dowData.__tablename__, engine, if_exists='replace',chunksize=50)

session.commit()

# Prep line chart data 
graph of citations given per day of month

# Remove all citations with value 'NO'

In [38]:
line = reduced_df[reduced_df.citationIssued == 'YES']
line.tail()

,OBJECTID,responseDate,citationIssued,lat,lon,gender,neighborhood,responseDow,responseDay,responseMonth,responseMonthName,responseYear
108901,108902,2018-12-31 06:03:24.000000,YES,44.991980,-93.294884,Male,Near - North,1,31,12,Dec,2018
108903,108904,2018-12-31 06:24:57.000000,YES,44.986340,-93.301460,Male,Near - North,1,31,12,Dec,2018
108907,108908,2018-12-31 07:24:56.000000,YES,45.000240,-93.284940,Male,Hawthorne,1,31,12,Dec,2018
108909,108910,2018-12-31 07:39:06.000000,YES,44.991970,-93.292590,Male,Near - North,1,31,12,Dec,2018
108919,108920,2018-12-31 10:05:58.000000,YES,44.949366,-93.262036,Male,Midtown Phillips,1,31,12,Dec,2018


# Keep only relevant columns

In [39]:
lines_df = line.drop(['OBJECTID', 'responseDow','lat', 'lon', 'gender'], axis=1)
lines_df['citationCnt'] = lines_df.groupby('responseDay')['citationIssued'].transform('count')
lines_df.head()

,responseDate,citationIssued,neighborhood,responseDay,responseMonth,responseMonthName,responseYear,citationCnt
60847,2018-01-01 05:00:03.000000,YES,Sheridan,01,01,Jan,2018,208
60853,2018-01-01 09:55:07.000000,YES,Lowry Hill East,01,01,Jan,2018,208
60886,2018-01-01 20:15:47.000000,YES,Downtown West,01,01,Jan,2018,208
60896,2018-01-02 02:32:10.000000,YES,Whittier,02,01,Jan,2018,267
60898,2018-01-02 03:17:47.000000,YES,Hawthorne,02,01,Jan,2018,267


# Add citation count column for graphing

In [40]:

lines_df= lines_df[['responseDay','citationCnt']]
lines_df=lines_df.drop_duplicates()

    

# Write out clean csv file

In [41]:
## Clean csv file to be saved in GitHub repository
lines_df.to_csv("copstop/static/data/clean_citation_data.csv", index=False,header=True)

# Add line data to database

In [42]:
lines_df.to_sql(citationData.__tablename__, engine, if_exists='replace',chunksize=50)

session.commit()

In [43]:
#test for data
# citation = session.query(citationData.neighborhood, citationData.citationCnt).filter_by(neighborhood="Como").first()

# citation

AttributeError: type object 'citationData' has no attribute 'neighborhood'